# Notebook showing basic operations of Kernel Memory - Document Ingestion

Prompt for secrets from the user and save in a local file.  Ensure that the local file is not checked
into source control.

In this case, the only secret is the Azure Open AI Key.

In [1]:
// Work-around for Console.Readline() not working in polyglot notebooks: https://github.com/dotnet/interactive/blob/main/docs/input-prompts.md

using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

string consoleInput(string prompt) {
    Task<string> inputTask = InteractiveKernel.GetInputAsync(prompt);
    string input = inputTask.Result;
    return input;
}

//Console.WriteLine($"Hello, {consoleInput("What's your name?")}!");


In [2]:
using System;

// Global configuration variables
var SECRETS_FILE = "azure_secrets.json";
var AZURE_OPENAI_ENDPOINT = "https://ragchatopenai.openai.azure.com/";
var POSTGRESS_SQL_ENDPOINT = "https://ragchatpostgres.postgres.database.azure.com/";
var OPENAI_MODEL = "gpt-4o";

string[] secretKeys = {"AZURE_OPENAI_KEY", "POSTGRES_SQL_PW"};

Dictionary<string, string> secrets = new Dictionary<string, string>();

if (System.IO.File.Exists(SECRETS_FILE)) {
    secrets = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(System.IO.File.ReadAllText(SECRETS_FILE));
} else {
    Console.WriteLine("Secrets file not found. Please enter the following secrets:");
    foreach (string key in secretKeys) {
        secrets[key] = consoleInput($"Please enter the value for {key}: ");
    }
    System.IO.File.WriteAllText(SECRETS_FILE, System.Text.Json.JsonSerializer.Serialize(secrets));
}

var AZURE_OPENAI_KEY = secrets["AZURE_OPENAI_KEY"];
var POSTGRES_SQL_PW = secrets["POSTGRES_SQL_PW"];


Import the required libraries and add a using statement.

In this case, the only required library is the Kernel Memory system.  Just get the latest stable version of the library (i.e. don't specify a version).

In [3]:
#r "nuget: Microsoft.KernelMemory.Core"

using Microsoft.KernelMemory;

Installed Packages Microsoft.KernelMemory.Core, 0.71.240820.1

Instantiate the framework.  Pass in two configurations - an embedding library to create the vector embeddings
using ADA002, and a configuration to ask questions of a LLM.  In this case, use GPT 4Omni.

In [4]:
var embeddingConfig = new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment = "Embedding",
    Endpoint = AZURE_OPENAI_ENDPOINT,
    APIKey = AZURE_OPENAI_KEY
};

var completionConfig = new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment = "GPT4o",
    Endpoint = AZURE_OPENAI_ENDPOINT,
    APIKey = AZURE_OPENAI_KEY
};

var kernelMemory = new KernelMemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration(config: embeddingConfig)
    .WithAzureOpenAITextGeneration(config: completionConfig)
    .Build();

warn: Microsoft.KernelMemory.AI.AzureOpenAI.AzureOpenAITextEmbeddingGenerator[0]
      Tokenizer not specified, will use GPT4Tokenizer. The token count might be incorrect, causing unexpected errors


Ingest a document.  This will create vector embeddings and store them in-memory.  This will store the full document broken up in chunk (snippets), with each chunk indexed by a vector describing the relevant aspects.  Kernel Memory then has the ability to search through the relevant chunks and only send the applicable ones to the LLM when a question is asked.

In [5]:
var docId = kernelMemory.ImportDocumentAsync(@".\documents\Recipies.docx").Result;
Console.WriteLine(docId);

be39a7b8f606472582dd74d6216b0227202408230410063440027


Ask multiple questions of the indexed memory.  For each question, the memory will retrieve relevant information from the vector store, and then send them to the LLM along with the question.  If no relevant information is found, the answer returned is: "INFO NOT FOUND" (this is configurable).

In [6]:
string[] questions =
{
    "Give me a pescatarian recipie for Sui Mai dumplings",
    "What's the difference between regular bread and no-knead bread?",
};

foreach (var question in questions)
{
    Console.WriteLine($"Question: {question}");
    var result = kernelMemory.AskAsync(question).Result;
    Console.WriteLine($"Response: {result}");
    Console.WriteLine("-------------------------------------------------------------");
}


Question: Give me a pescatarian recipie for Sui Mai dumplings
Response: ### Pescatarian Sui Mai Dumplings

#### Ingredients:
- 1 lb vegan sausage (14 oz package of Lightlife sausage is particularly good, can also use a combination of vegan sausage and shrimp or scallops)
- 2/3 can of water chestnuts, chopped (open a can and eat a couple, chop the rest)
- 1 tbsp minced ginger
- 1 tbsp cornstarch
- 1 tbsp soy sauce
- 1 tbsp sugar
- 1 tsp toasted sesame oil
- 1 tbsp peanut or canola oil
- 1/4 cup chopped cilantro
- 1 chopped green onion
- 40 wonton skins (or egg roll wrappers cut in 4ths)

#### Instructions:
1. **Prepare the Filling:**
   - In a large bowl, mix together the vegan sausage, chopped water chestnuts, minced ginger, cornstarch, soy sauce, sugar, toasted sesame oil, peanut or canola oil, chopped cilantro, and chopped green onion until well combined.

2. **Assemble the Dumplings:**
   - Place a wonton skin on a clean surface.
   - Add about 1 tablespoon of the filling to the cen

Search Memory for relevant results based on the question.  This will return full snippets of the source document which are relevant, along with a score which indicates the relevancy.

In [7]:
var response = kernelMemory.SearchAsync("How do I make dolmades?").Result;
response.Display();


Microsoft.KernelMemory.SearchResult Query How do I make dolmades? NoResult False Results index value 0 Microsoft.KernelMemory.Citation Link default/be39a7b8f606472582dd74d6216b0227202408230410063440027/def269cb0e6644b9ad36ad41dd35dbac Index default DocumentId be39a7b8f606472582dd74d6216b0227202408230410063440027 FileId def269cb0e6644b9ad36ad41dd35dbac SourceContentType application/vnd.openxmlformats-officedocument.wordprocessingml.document SourceName Recipies.docx SourceUrl /download?index=default&documentId=be39a7b8f606472582dd74d6216b0227202408230410063440027&filename=Recipies.docx Partitions index value 0 Microsoft.KernelMemory.Citation+Partition Text Make the filling: Saute onions in 2 tbs oil for 15 minutes until browned and slightly caramelized. Add onions, spinach, olive oil, salt and pepper to the potato mash.
Assemble the knishes: Roll out dough as thin as possible into ~14"/6" rectangle. Add filling to about 1" from the edges. Fold the dough over long-ways to form a log. Seal the outside edge. Place seam side down on lightly oiled backing dish. Score into 5ths and brush with olive oil 5.Cook: Bake in preheated 350 oven for 40 minutes
Dolmades
Saturday, March 6, 202110:00 AM
A variant of Judy's recipe, but with chick peas instead of ground beef, a bit more rice, and allspice. Yum!
Stuffing:
1 can chick peas, drained and rinced
1 onion, chopped
2 cups uncooked white rice (or even better; 1 packet Costco quinoa and brown rice + 1/2 cup uncooked white rice)
1/2+ tsp cinnamon
1/2+ tsp allspice
1/2+ tsp salt

1 can diced tomatoes (16 oz), including juice, or 1 8oz can tomato sauce 1 16 oz jar grape leaves (56 leaves)
Sauce:
4 eggs
2 large lemons
Directions:
Mix stuffing ingredients
Put a small amount in each grape leaf and wrap like a burrito
Place snuggly in medium sized pot, add 4 cups water, place heavy lid directly on dolmades
Cook for 1 hour on low heat, adding water to have 2 cups at the end (likely will need another 2 cups)
Remove from heat
Make the sauce by beating the eggs, pour in 2 cups hot dolmade broth slowly while continuing to whisk, and then add lemon juice (basically like hollandaise sauce with broth instead of butter) 7.Pour sauce over dolmades and serve immediately
Sherry's beans
Thursday, May 27, 202112:58 PM
1 onion, chopped
(optional) 4 slices turkey bacon, cut into bite sized pieces
4 cans of beans, drained and rinsed
1/2 cup brown sugar
1/4 tsp nutmeg
1- 15oz can chopped tomato w juice 1- 8oz can tomato sauce
Directions:
(optional) Fry bacon until crisp, set aside
Saute onions until caramelized
Add the rest of the ingredients and cook on low heat, uncovered, for an hour, stirring occasionally

My Eggs Benedict
Saturday, May 29, 20218:47 AM
Serves 2
2 egg yolks
1 tbs lemon juice
1/8 tsp dijon (optional)
5 tbs butter
4 eggs
Smoked salmon
For starch, either toasted English muffins, roasted potatoes, or slices of fresh baked bread.
Directions:
Make the Holendaise sauce
Whisk together egg yokes, lemon juice, and optionally the dijon
Heat butter in micro wave on high in a covered dish for a full minute, to get the butter very hot
Pour butter into egg yokes while continuing to whisk
In parallel, cook the eggs over easy (so much simpler than poaching and it tastes better). Optionally add the egg whites left over from making the Holandaise sauce.
Assemble the layers on plates and serve; starch, lox, eggs, Holendaise sauce
Buffalo wings
Monday, June 7, 20216:38 PM
Vegan chicken strips, cooked per package.
Buffalo Sauce; equal parts butter and hot sauce (e.g. franks red hot), heated and mixed together. Optionally add vinegar, Worcestershire, garlic, cayenne, honey, mustard to taste (not necessary).
Coat chicken strips with sauce, serve with carrots, celery, and blue cheese dressing.
Optionally put chicken and sauce and blue cheese on a bun.

Portobello burgers
Thursday, June 10, 20212:54 AM 4 portobello caps, rinsed
Marinade:
2tbs balsamic
1tbs light soy
1tbs olive oil
Meat seasoning (burbon and brown sugar)

Create a new instance of Kernel Memory, this time configuring it to use PostgreSQL as the store for vectors.

In [8]:
var connString = $"Host=salearningpostgres.postgres.database.azure.com;Database=ragchat0;Username=SALearningAdmin;Password={POSTGRES_SQL_PW};SSLMode=Require;Trust Server Certificate=true";

// AI Kernel Memory - Persist vectors to Postgres
var postConfig = new PostgresConfig {
    ConnectionString = connString
};

var Endpoint = "https://ragchatopenai.openai.azure.com/";

var kernelMemoryDB = new KernelMemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration(config: embeddingConfig)
    .WithAzureOpenAITextGeneration(config: completionConfig)
    .WithPostgresMemoryDb(config: postConfig)
    .Build();

Ingest all files, in Launch directory, this time saving into database.

In [9]:
var docId = kernelMemoryDB.ImportDocumentAsync(@".\documents\Recipies.docx").Result;
Console.WriteLine(docId);

8b0494d59b5b4c928db54868f9264136202408230411151793522


Ask question, this time using the saved vector embeddings in the database as opposed to the in memory store.

In [10]:
foreach (var question in questions)
{
    Console.WriteLine($"Question: {question}");
    var result = kernelMemoryDB.AskAsync(question).Result;
    Console.WriteLine($"Response: {result}");
    Console.WriteLine("-------------------------------------------------------------");
}



Question: Give me a pescatarian recipie for Sui Mai dumplings
Response: ### Pescatarian Sui Mai Dumplings

#### Ingredients:
- 14 oz package of vegan sausage (Lightlife sausage is particularly good)
- 1/2 lb shrimp or scallops, chopped
- 2/3 can of water chestnuts, chopped (open a can and eat a couple, chop the rest)
- 1 tbsp minced ginger
- 1 tbsp cornstarch
- 1 tbsp soy sauce
- 1 tbsp sugar
- 1 tsp toasted sesame oil
- 1 tbsp peanut or canola oil
- 1/4 cup chopped cilantro
- 1 chopped green onion
- 40 wonton skins (or egg roll wrappers cut in 4ths)

#### Instructions:
1. **Prepare the Filling:**
   - In a large bowl, mix together the vegan sausage, chopped shrimp or scallops, chopped water chestnuts, minced ginger, cornstarch, soy sauce, sugar, toasted sesame oil, peanut or canola oil, chopped cilantro, and chopped green onion until well combined.

2. **Assemble the Dumplings:**
   - Place about 1 tablespoon of the filling in the center of each wonton skin.
   - Pinch the skin to alm